In [1]:

#Import Packages
import pandas as pd
import requests
from pandas.io.json import json_normalize
import datetime
import math
from dateutil.relativedelta import relativedelta
from pandasql import sqldf
import config


In [2]:

#Global
pysqldf = lambda q: sqldf(q, globals())
now = datetime.date.today()
pd.set_option('display.float_format', '{:.4f}'.format)


In [3]:

#Token
token_url = 'https://www.ura.gov.sg/uraDataService/insertNewToken.action'
AccessKey = config.AccessKey
#http://httpbin.org/get to get user-agent
useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
tokenheaders = {'AccessKey': AccessKey, 'User-Agent':useragent }
tokenreq =requests.get(token_url, headers= tokenheaders).json()
TokenKey = tokenreq['Result']
apiheaders = {'AccessKey': AccessKey, 'Token': TokenKey, 'User-Agent':useragent }

In [63]:

#GET API - private transaction
privatetransaction_url1 = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=1'
privatetransaction_url2 = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=2'
privatetransaction_url3 = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=3'
privatetransaction_url4 = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=4'


batch1req = requests.get(privatetransaction_url1, headers= apiheaders).json()
batch2req = requests.get(privatetransaction_url2, headers= apiheaders).json()
batch3req = requests.get(privatetransaction_url3, headers= apiheaders).json()
batch4req = requests.get(privatetransaction_url4, headers= apiheaders).json()


In [20]:

#GET API - rental transaction
rentaltransaction_url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod='
#get past 5 years
# refperiod = str(now.year)[-2:] + 'q'+str(math.floor(now.month/3))
#input vlaue
refperiod = '21q4'
rentalreq = requests.get(rentaltransaction_url+refperiod, headers= apiheaders).json()
df_rental = json_normalize(data=rentalreq['Result'], record_path='rental', 
                            meta=['street','project'])                          

In [65]:

#GET API - developer transaction
developer_url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Developer_Sales&refPeriod='
#cuumlative 3 years as of date
# if (now.month - 1 < 10):
#     refperiod2 = '0'+str(now.month-1)+ str(now.year)[-2:] 
# else:
#      refperiod2 = str(now.month-1)+ str(now.year)[-2:] 
refperiod2 = '0122'
developerreq = requests.get(developer_url+refperiod2, headers= apiheaders).json()


{'Status': 'Success',
 'Result': [{'street': 'LORONG H TELOK KURAU',
   'district': '15',
   'propertyType': 'Non-Landed',
   'project': 'RYMDEN 77',
   'developer': 'QHS Development Pte Ltd',
   'developerSales': [{'refPeriod': '0122',
     'medianPrice': 1753,
     'highestPrice': 1787,
     'lowestPrice': 1751,
     'launchedToDate': 31,
     'soldInMonth': 3,
     'launchedInMonth': 0,
     'soldToDate': 7,
     'unitsAvail': 31}],
   'marketSegment': 'OCR'},
  {'street': 'SILAT AVENUE',
   'district': '03',
   'propertyType': 'Non-Landed',
   'project': 'AVENUE SOUTH RESIDENCE',
   'developer': 'United Venture Development (Silat) Pte Ltd',
   'developerSales': [{'refPeriod': '0122',
     'medianPrice': 2312,
     'highestPrice': 2423,
     'lowestPrice': 2024,
     'launchedToDate': 1000,
     'soldInMonth': 13,
     'launchedInMonth': 0,
     'soldToDate': 926,
     'unitsAvail': 1074}],
   'marketSegment': 'RCR'},
  {'street': 'DE SOUZA AVENUE',
   'district': '21',
   'property

In [66]:

#GET API - latest in pipeline 
pipeline_url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Pipeline'
#Update Frequency: End of day of every 4th Friday of January, April, July and October
pipelinereq = requests.get(pipeline_url, headers= apiheaders).json()

In [82]:
#Transform to dataframe - private transaction
df_batch1 = json_normalize(data=batch1req['Result'], record_path='transaction', 
                            meta=['street','project', 'marketSegment'])
df_batch2 = json_normalize(data=batch2req['Result'], record_path='transaction', 
                            meta=['street','project', 'marketSegment'])
df_batch3 = json_normalize(data=batch3req['Result'], record_path='transaction', 
                            meta=['street','project', 'marketSegment'])
df_batch4 = json_normalize(data=batch4req['Result'], record_path='transaction', 
                            meta=['street','project', 'marketSegment'])
df_private = df_batch1.append([df_batch2, df_batch3,df_batch4])

In [21]:
#Transform to dataframe - rental transaction
df_rental = json_normalize(data=rentalreq['Result'], record_path='rental', 
                            meta=['street','project'])

,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,street,project
5090,120-130,1221,Non-landed Properties,05,1300-1400,2,3550,JALAN LEMPENG,THE TRILINQ
21561,80-90,1021,Non-landed Properties,09,800-900,2,4750,CAIRNHILL ROAD,THE LAURELS
11355,90-100,1121,Non-landed Properties,05,1000-1100,3,4500,WEST COAST CRESCENT,SEAHILL
20686,120-130,1021,Non-landed Properties,17,1300-1400,3,3000,FLORA ROAD,AZALEA PARK CONDOMINIUM
13927,80-90,1121,Non-landed Properties,21,900-1000,2,2400,HINDHEDE WALK,SPRINGDALE CONDOMINIUM


In [83]:
#Transform to dataframe - developer transaction 
df_developer = json_normalize(data=developerreq['Result'], record_path='developerSales', 
                            meta=['street','district','propertyType','project','developer','marketSegment'])


,refPeriod,medianPrice,highestPrice,lowestPrice,launchedToDate,soldInMonth,launchedInMonth,soldToDate,unitsAvail,street,district,propertyType,project,developer,marketSegment
0,0122,1753,1787,1751,31,3,0,7,31,LORONG H TELOK KURAU,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,OCR
1,0122,2312,2423,2024,1000,13,0,926,1074,SILAT AVENUE,03,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,RCR
2,0122,1787,1890,1725,258,16,0,208,258,DE SOUZA AVENUE,21,Non-Landed,VERDALE,C&C (JJK) Pte Ltd,RCR
3,0122,1768,1974,1507,1310,27,0,1274,1410,HOUGANG AVENUE 2,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,OCR
4,0122,2577,2609,2544,160,2,0,113,319,HOLLAND ROAD,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,CCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0122,0,0,0,0,0,0,0,24,LORONG 13 GEYLANG,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,RCR
149,0122,2158,2371,2005,667,34,0,595,667,BIDADARI PARK DRIVE,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Mall Pte Ltd/The Woodleigh Resid...,RCR
150,0122,3558,3558,3558,500,1,0,406,558,TAN QUEE LAN STREET,07,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,CCR
151,0122,1531,1593,1456,358,4,22,334,448,CANBERRA DRIVE,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,OCR


In [70]:

#Transform to dataframe - latest in pipeline 
df_pipeline = json_normalize(data=pipelinereq['Result'])
df_pipeline.sample(5)

,noOfApartment,expectedTOPYear,noOfSemiDetached,street,district,project,noOfDetachedHouse,noOfTerrace,noOfCondo,developerName,totalUnits
50,1399,na,0,Sims Avenue,14,Parc Esta,0,0,0,MCL Land (Everbright) Pte Ltd,1399
38,407,na,0,Northumberland Road,08,Piccadilly Grand,0,0,0,Maximus Residential SG Pte Ltd/Maximus Commerc...,407
66,716,na,0,West Coast Vale,05,Whistler Grand,0,0,0,CDL Pegasus Pte Ltd,716
56,1012,na,0,Serangoon North Avenue 1,19,Affinity At Serangoon,0,40,0,Oxley Serangoon Pte Ltd,1052
43,154,na,0,Orchard Boulevard,10,Boulevard 88,0,0,0,Granmil Holdings Pte Ltd,154


In [84]:

#Format Data
df_private['contractDate'] = '01' +df_private['contractDate'] 
df_rental['leaseDate']  = '01' +df_rental['leaseDate'] 
df_developer['refPeriod']  = '01' +df_developer['refPeriod']
df_private['contractDate'] = df_private['contractDate'].apply(lambda x: datetime.datetime.strptime(x, '%d%m%y').strftime('%d-%m-%y'))
df_rental['leaseDate'] = df_rental['leaseDate'] .apply(lambda x: datetime.datetime.strptime(x, '%d%m%y').strftime('%d-%m-%y'))
df_developer['refPeriod'] = df_developer['refPeriod'].apply(lambda x: datetime.datetime.strptime(x, '%d%m%y').strftime('%d-%m-%y'))

In [87]:

#Add Data
def fourth_friday(year, month):
    """Return datetime.date for monthly option expiration given year and
    month
    """
    # The 15th is the lowest third day in the month
    fourth = datetime.date(year, month, 28)
    # What day of the week is the 15th?
    w = fourth.weekday()
    # Friday is weekday 4
    if w != 4:
        # Replace just the day (of month)
        fourth = fourth.replace(day=(28 + (4 - w) % 7))
    return fourth
#Update Frequency: End of day of every 4th Friday of January, April, July and October
jandate = fourth_friday(now.year,1)
aprdate = fourth_friday(now.year,4)
juldate = fourth_friday(now.year,7)
octdate = fourth_friday(now.year,10)

if now >= octdate:
   datadate = octdate
elif now >= juldate:
    datadate = juldate
elif now >= aprdate:
    datadate = aprdate
else:
    datadate = jandate
df_pipeline['datadate'] = datadate

# Get Tenure details 
df_private['TenureType'] = df_private['tenure'].apply(lambda x: x if x == 'Freehold' else 'Leasehold')
df_private['BuildYear'] = df_private['tenure'].apply(lambda x: None if x == 'Freehold' else str(x)[-4:])
df_private['LeaseYear'] = df_private['tenure'].apply(lambda x: None if x == 'Freehold' else \
    (str(x)[:6][:2] if str(x)[:6][-3:] == 'yrs' else \
        (str(x)[:6][:3] if str(x)[:6][-2:] == 'yr' else\
            (str(x)[:6][:4] if str(x)[:6][-1:] == 'y' else '999999'))))


In [89]:

#Keep old version
df_privatedatabase = pd.read_excel('Data/TransactionDatabase.xlsx')
df_privatedatabase.drop(df_privatedatabase.filter(regex="Unnamed"),axis=1, inplace=True)
df_privatedatabase.to_excel('History/TransactionDatabaseOld.xlsx')
df_rentaldatabase = pd.read_excel('Data/RentalDatabase.xlsx')
df_rentaldatabase.drop(df_rentaldatabase.filter(regex="Unnamed"),axis=1, inplace=True)
df_rentaldatabase.to_excel('History/RentalDatabaseOld.xlsx')
df_developerdatabase = pd.read_excel('Data/DeveloperDatabase.xlsx')
df_developerdatabase.drop(df_developerdatabase.filter(regex="Unnamed"),axis=1, inplace=True)
df_developerdatabase.to_excel('History/DeveloperDatabaseOld.xlsx')

In [90]:

#Append Database
# TO CHECK!!!!! in next append
#Private Transactions
# update 61 months; keep data > 61 months
df_private['contractDate'] = pd.to_datetime(df_private['contractDate'],errors='coerce')
last_date = df_private['contractDate'].max()
max_date = last_date + relativedelta(years = -5)
df_privatedatabase['contractDate'] = pd.to_datetime(df_privatedatabase['contractDate'],errors='coerce')
df_historical = df_privatedatabase[(df_privatedatabase['contractDate'] >= max_date)]
df_privatedatabaseappend = df_historical.append(df_private).drop_duplicates()


#Rental Transactions
# update 61 months; keep data > 61 months
df_rental['leaseDate'] = pd.to_datetime(df_rental['leaseDate'],errors='coerce')
last_date = df_rental['leaseDate'].max()
max_date = last_date + relativedelta(years = -5)
df_renthistorical = df_rentaldatabase[(df_rentaldatabase['leaseDate'] >= max_date)]
df_rentaldatabaseappend = df_renthistorical.append(df_rental).drop_duplicates()


#Developer 
#Keep 2 years of data
df_developer['refPeriod'] = pd.to_datetime(df_developer['refPeriod'] ,errors='coerce')
max_date_dev = df_developer['refPeriod'].max()  + relativedelta(years = -2)
df_developerdatabase['refPeriod'] = pd.to_datetime(df_developerdatabase['refPeriod'],errors='coerce')
df_developerhistorical = df_developerdatabase[(df_developerdatabase['refPeriod'] >= max_date_dev)]
df_developerdatabaseappend = df_developerhistorical.append(df_developer).drop_duplicates()


/Users/opx/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [91]:

#Export Data
df_privatedatabaseappend.to_excel('Data/TransactionDatabase.xlsx')
df_rentaldatabaseappend.to_excel('Data/RentalDatabase.xlsx')
df_developerdatabaseappend.to_excel('Data/DeveloperDatabase.xlsx')
df_pipeline.to_excel('Data/Pipeline.xlsx')